# オブジェクト指向プログラミング

# Challenge
* 銀行口座のAccountクラスを作る
* 残高(balance)と口座名を元に口座(Account)を作る
* withdrawメソッドとdepositメソッドで残高を変更する
* 残高が足りなければ引き落とせないようにする
* 口座番号(account_number)は今まで作成された口座のIDとなるように連番をふる
* 残高が変更されたら、口座名、口座番号とその残高を表示する

## リファクタ前

In [20]:
class Account:
    account_number = 0

    def __init__(self, name, balance):
        self.name = name
        self.balance = balance
        Account.account_number += 1

    def display_account_info(self, process_name):
        print(f"口座番号：{Account.account_number}, {process_name}後残高：{self.balance}")

    def withdraw(self, amount):
        process_name = "出金"
        if self.balance < amount:
            print("残高が不足しています")
        else:
            self.balance -= amount
            self.display_account_info(process_name)

    def deposit(self, amount):
        process_name = "入金"
        self.balance += amount
        self.display_account_info(process_name)

## リファクタ後
* account_numberをクラス変数に持っていればよいのでは？(リファクタ前)
  * account_numberは、"その口座のID"。クラスに紐づくべきではなく、インスタンスにひもづくべき
  * クラス変数にはあくまでも今までいくつ口座が作成されたかを示す"count"を保持する

In [39]:
class Account:
    count = 0

    def __init__(self, name, balance):
        self.name = name
        self.balance = balance
        self.account_number = Account.count
        Account.count += 1

    def display_account_info(self, process_name):
        print(f"口座番号：{self.account_number}, {process_name}後残高：{self.balance}")

    def withdraw(self, amount):
        process_name = "出金"
        if self.balance < amount:
            print("残高が不足しています")
        else:
            self.balance -= amount
            self.display_account_info(process_name)

    def deposit(self, amount):
        process_name = "入金"
        self.balance += amount
        self.display_account_info(process_name)

In [30]:
taro = Account("taro", 10000)
taro.deposit(5000)
taro.withdraw(3000)
jiro = Account("jiro", 5000)
jiro.withdraw(10000)
jiro.deposit(100000)

口座番号：0, 入金後残高：15000
口座番号：0, 出金後残高：12000
残高が不足しています
口座番号：1, 入金後残高：105000


# スタティックメソッド
* インスタンスに紐づかないメソッド
* @staticmethodデコレータを使う
* 主にクラス内の便利関数のように使用する
* 引数にselfを取らない(インスタンスの情報は使わないので)
  * クラスの情報も使用しない
* クラスからアクセスしてcallする
* クラスの情報を使う場合はクラスメソッドを使う

In [37]:
class MyClass:
    def mymethod(self):
        print("This is normal method! from {}".format(self))

    @staticmethod
    def mystaticmethod():
        print("This is staticmethod!")

In [38]:
c = MyClass()
c.mymethod()
MyClass.mystaticmethod()
c.mystaticmethod()  # 一応これでも呼べるが使わない。インスタンスの中のメソッドを先に探して、次にstaticnethodやclassmethodを探す

This is normal method! from <__main__.MyClass object at 0x111628110>
This is staticmethod!
This is staticmethod!


## Challenge
* 前回作成したAccountクラスに、取引(transaction)を記録する仕組みを追加する
* 取引として保持する情報は、
  * "withdraw/depositの金額"
  * "新しい残高"
  * "日時"
* それぞれの取引情報をdictionaryとして保持し、それをlistでインスタンス変数で保持すればOK
* 日時を作る関数をstaticmethodで作ってみよう

## リファクタ前

In [55]:
from datetime import datetime as dt

In [53]:
dt.now().strftime("%d/%m/%Y, %H:%M:%S")

'21/01/2024, 13:35:51'

In [56]:
class Account:
    count = 0
    transaction_list = []

    def __init__(self, name, balance):
        self.name = name
        self.balance = balance
        self.account_number = Account.count
        Account.count += 1

    @staticmethod
    def current_datetime():
        return dt.now()

    def display_account_info(self, process_name):
        print(f"口座番号：{self.account_number}, {process_name}後残高：{self.balance}")

    # def transaction(self):
    #     transaction_dict = {}

    def withdraw(self, amount):
        process_name = "出金"
        transaction_dict = {}
        if self.balance < amount:
            print("残高が不足しています")
        else:
            self.balance -= amount
            transaction_dict["user"] = self.name
            transaction_dict["amount"] = -amount
            transaction_dict["balance"] = self.balance
            transaction_dict["datetime"] = Account.current_datetime()
            Account.transaction_list.append(transaction_dict)
            self.display_account_info(process_name)

    def deposit(self, amount):
        transaction_dict = {}
        process_name = "入金"
        self.balance += amount
        transaction_dict["user"] = self.name
        transaction_dict["amount"] = amount
        transaction_dict["balance"] = self.balance
        transaction_dict["datetime"] = Account.current_datetime()
        Account.transaction_list.append(transaction_dict)
        self.display_account_info(process_name)

In [57]:
taro = Account("taro", 10000)
taro.deposit(5000)
taro.withdraw(3000)
jiro = Account("jiro", 5000)
jiro.withdraw(10000)
jiro.deposit(100000)

口座番号：0, 入金後残高：15000
口座番号：0, 出金後残高：12000
残高が不足しています
口座番号：1, 入金後残高：105000


In [58]:
Account.transaction_list

[{'user': 'taro',
  'amount': 5000,
  'balance': 15000,
  'datetime': datetime.datetime(2024, 1, 21, 13, 39, 3, 345366)},
 {'user': 'taro',
  'amount': -3000,
  'balance': 12000,
  'datetime': datetime.datetime(2024, 1, 21, 13, 39, 3, 345441)},
 {'user': 'jiro',
  'amount': 100000,
  'balance': 105000,
  'datetime': datetime.datetime(2024, 1, 21, 13, 39, 3, 345531)}]

## リファクタ後
* 各ユーザーの取引履歴をインスタンス変数として保持するイメージ
  * 自分が書いたのは、システム全体の取引テーブルに記録するイメージのみ

In [77]:
class Account:
    count = 0

    def __init__(self, name, balance):
        self.name = name
        self.balance = balance
        self.account_number = Account.count
        self.transaction_history = []
        Account.count += 1

    @staticmethod
    def get_current_datetime():
        return dt.now().strftime("%Y-%m-%d %H:%M:%S")

    def display_account_info(self, process_name):
        print(f"口座番号：{self.account_number}, {process_name}後残高：{self.balance}")

    def withdraw(self, amount):
        process_name = "出金"
        if self.balance < amount:
            print("残高が不足しています")
        else:
            self.balance -= amount
            self.add_transaction_history(-amount)
            self.display_account_info(process_name)

    def deposit(self, amount):
        process_name = "入金"
        self.balance += amount
        self.add_transaction_history(amount)
        self.display_account_info(process_name)

    def add_transaction_history(self, amount):
        transaction = {
            "withdraw/deposit": amount,
            "new_balance": self.balance,
            "datetime": Account.get_current_datetime(),
        }
        self.transaction_history.append(transaction)

    def show_transaction_history(self):
        for transaction in self.transaction_history:
            transaction_str_list = []
            for k, v in transaction.items():
                transaction_str_list.append(f"{k}: {v}")
            print(", ".join(transaction_str_list))

In [78]:
taro = Account("taro", 10000)
taro.deposit(5000)
taro.withdraw(3000)
jiro = Account("jiro", 5000)
jiro.withdraw(10000)
jiro.deposit(100000)
taro.show_transaction_history()

口座番号：0, 入金後残高：15000
口座番号：0, 出金後残高：12000
残高が不足しています
口座番号：1, 入金後残高：105000
withdraw/deposit: 5000, new_balance: 15000, datetime: 2024-01-21 14:00:58
withdraw/deposit: -3000, new_balance: 12000, datetime: 2024-01-21 14:00:58


# クラスメソッド
* インスタンスに紐づかないメソッド
* @classmethodデコレータを使う
* 主にクラス内で便利関数のように使用する
* **引数にclsを取って、クラスの情報にアクセスできる**
* クラスからアクセスしてcallする
* クラスメソッド内でインスタンスを生成して返すことも可能(ファクトリーメソッド)

In [84]:
class MyClass:
    classmethod_count = 0

    def mymethod(self):
        print("This is normal method! from {}".format(self))

    @staticmethod
    def mystaticmethod():
        print("This is staticmethod!")

    @classmethod
    def myclassmethod(cls):
        cls.classmethod_count += 1
        print(f"This is classmethod and now thoe count is {cls.classmethod_count}")

In [85]:
c = MyClass()
c.mystaticmethod()
MyClass.myclassmethod()

This is staticmethod!
This is classmethod and now thoe count is 1


### ファクトリーメソッド

In [90]:
from datetime import datetime as dt


class Person:
    def __init__(self, name, age) -> None:
        self.name = name
        self.age = age

    @classmethod
    def create_dob_from_age(cls, name, year, month, date):
        today = dt.now()
        # if today.month < month and today.date < date:
        if (today.month, today.date) < (year, date):
            age = today.year - year - 1
        else:
            age = today.year - year
        return cls(name=name, age=age)

In [92]:
john = Person("john", 20)
emma = Person.create_dob_from_age("Emma", 1989, 4, 3)
print(emma.name)
print(emma.age)

Emma
34


# private変数と名前修飾(name mangling)
* private変数は、クラス外からアクセスできない変数
* Pythonには「private変数」はない
* 変数名の接頭辞に「_」をt蹴ることで、non publicにすることができる
  * あくまでも慣習であり、強制力はない
* 「__」(アンスコ2つ)をつけることで、名前修飾する
  * 名前就職された変数名は、_<Class>__<attribute>のような形になる
    * 例. _Account__balance
  * 結果、private変数のような役割をつけることができる

In [3]:
class Account:
    def __init__(self, balance) -> None:
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount
        self.show_balance()

    def withdraw(self, amount):
        if amount > self.balance:
            print("残高が不足しています")
        else:
            self.balance -= amount
            self.show_balance()

    def show_balance(self):
        print(f"残高は{self.balance}円です")

In [5]:
my_account = Account(10000)
my_account.deposit(3000)
my_account.withdraw(5000)

# 下記も実行できてしまう
my_account.balance = -1000
print(my_account.balance)

残高は、13000円です
残高は、8000円です
-1000


## non public化
* 慣習であり、あくまでも意思表示

In [6]:
class Account:
    def __init__(self, balance) -> None:
        self._balance = balance

    def deposit(self, amount):
        self._balance += amount
        self.show_balance()

    def withdraw(self, amount):
        if amount > self._balance:
            print("残高が不足しています")
        else:
            self._balance -= amount
            self.show_balance()

    def show_balance(self):
        print(f"残高は{self._balance}円です")

In [8]:
my_account = Account(10000)
my_account.deposit(3000)
my_account.withdraw(5000)
# 下記は実行できて、新たに"balance"変数に-1000がセットされる
my_account.balance = -1000
print(my_account.balance)
print(my_account._balance)

残高は13000円です
残高は8000円です
-1000
8000


## 名前修飾

In [10]:
class Account:
    def __init__(self, balance) -> None:
        self.__balance = balance

    def deposit(self, amount):
        self.__balance += amount
        self.show_balance()

    def withdraw(self, amount):
        if amount > self.__balance:
            print("残高が不足しています")
        else:
            self.__balance -= amount
            self.show_balance()

    def show_balance(self):
        print(f"残高は{self.__balance}円です")

In [15]:
my_account = Account(10000)
print(dir(my_account))

['_Account__balance', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'deposit', 'show_balance', 'withdraw']


In [16]:
my_account.deposit(3000)
my_account.withdraw(5000)
print(my_account.__balance)

残高は13000円です
残高は8000円です


AttributeError: 'Account' object has no attribute '__balance'

In [18]:
# 下記は実行できて、名前修飾されたbalanceとは別に、新たに__balanceにセットされる
my_account.__balance = -1000
my_account.show_balance()

残高は8000円です


In [19]:
# 下記は実行できてしまう。アクセス制御されるわけではない
my_account._Account__balance = -1000
my_account.show_balance()

残高は-1000円です


# getterとsetter
* 直接読みに行ったり、値のバリデーションを行うために使用する

In [20]:
class Person:
    def __init__(self, name, age) -> None:
        self.name = name
        self.age = age

In [33]:
# インスタンス変数を取得したり、変更するのを関数化する
class Person:
    def __init__(self, name, age) -> None:
        self.name = name
        self.age = age

    def get_age(self):
        print("get age called!!")
        return self.age

    def set_age(self, age):
        print("set_age called!!")
        if age < 0:
            print("0以上の値を入れてください")
        else:
            self.age = age

In [34]:
john = Person("john", 15)
# print(john.age)
print(john.get_age())
# john.age = 15
john.set_age(20)
print(john.get_age())

get age called!!
15
set_age called!!
get age called!!
20


In [35]:
# getterとsetterに書き換え
class Person:
    def __init__(self, name, age) -> None:
        self.name = name
        self._age = age

    def get_age(self):
        print("get age called!!")
        return self._age

    def set_age(self, age):
        print("set_age called!!")
        if age < 0:
            print("0以上の値を入れてください")
        else:
            self._age = age

    age = property(get_age, set_age)

In [38]:
john = Person("john", 15)
john.age  ## 通常なら直接変数を読みに行くのだが、getterが動いている
john.age = 20  ## 通常なら直接変数を読みに行くのだが、getterが動いている

get age called!!
set_age called!!


## プロパティデコレータ

In [39]:
# デコレータを使用したパイソニックな書き方にリファクタ
class Person:
    def __init__(self, name, age) -> None:
        self.name = name
        self._age = age

    @property
    def age(self):
        print("get age called!!")
        return self._age

    @age.setter
    def age(self, age):
        print("set_age called!!")
        if age < 0:
            print("0以上の値を入れてください")
        else:
            self._age = age

    # age = property(get_age, set_age)

In [40]:
john = Person("john", 15)
john.age  ## 通常なら直接変数を読みに行くのだが、getterが動いている
john.age = 20  ## 通常なら直接変数を読みに行くのだが、getterが動いている

get age called!!
set_age called!!


# 継承
* 他のクラスをベースのクラスとして継承して、別のクラスを作ることができる
* super class(親クラス、基底クラス)の機能を引き継ぎ、sub class(子クラス、派生クラス)として拡張することができる

In [46]:
class Animal(object):
    def __init__(self, name) -> None:
        self.name = name
        print("Animal init is called!!")

    def breath(self):
        print(f"{self.name} is breathing")


class Dog(Animal):
    pass


class Cat(Animal):
    pass

In [47]:
# DogとCatのインスタンスを作成した段階で、Animalのinitが実行される
pochi = Dog("pochi")
tama = Cat("tama")
pochi.breath()
tama.breath()

Animal init is called!!
Animal init is called!!
pochi is breathing
tama is breathing


## 継承時のコンストラクタ(init)
* sub classにコンストラクタがない場合、super classのinitが呼ばれる
* sub classにコンストラクタがある場合、sub classのinitが優先される
  * ただし、sub classのinitでsuper classのinitを使用することができる　super()

In [49]:
# エラー例
class Animal(object):
    def __init__(self, name) -> None:
        self.name = name
        print("Animal init is called!!")

    def breath(self):
        print(f"{self.name} is breathing")


class Dog(Animal):
    def __init__(self) -> None:
        print("Dog is called!!")


class Cat(Animal):
    pass


pochi = Dog("pochi")

TypeError: Dog.__init__() takes 1 positional argument but 2 were given

In [51]:
class Animal(object):
    def __init__(self, name) -> None:
        self.name = name
        print("Animal init is called!!")

    def breath(self):
        print(f"{self.name} is breathing")


class Dog(Animal):
    def __init__(self, name) -> None:
        # self.name = name 親と同じこと書いている
        super().__init__(name=name)
        print("Dog is called!!")


class Cat(Animal):
    pass


pochi = Dog("pochi")

Animal init is called!!
Dog is called!!


## Challenge